# Model Deployment - Complete Guide

## 📚 Learning Objectives
- Save and load trained models
- Create prediction functions
- Build a simple API with Flask
- Implement model versioning
- Handle production considerations
- Monitor model performance

## 🎯 What is Model Deployment?

**Model Deployment** is the process of making your trained machine learning model available for use in a production environment.

### Deployment Pipeline:
```
Training → Validation → Saving → API Creation → Deployment → Monitoring
```

### Why is Deployment Important?
- A model is only valuable if it can be used
- Real-world impact requires production deployment
- Enables continuous improvement through feedback

### Deployment Options:
1. **Batch Predictions**: Scheduled predictions on datasets
2. **Real-time API**: On-demand predictions via REST API
3. **Embedded**: Model integrated into applications
4. **Edge Deployment**: Models on mobile/IoT devices

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import pickle
import joblib
import json
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")
print(f"Current timestamp: {datetime.now()}")

## Part 1: Model Training and Saving
### 1️⃣ Train a Production-Ready Model

In [ ]:
# Load data
df = pd.read_csv('supervised Learning/01_Regression/Linear Regression/data/dataset.csv')

# Prepare features and target
target_col = 'median_house_value'
feature_cols = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
                'total_bedrooms', 'population', 'households', 'median_income']

X = df[feature_cols].fillna(df[feature_cols].median())
y = df[target_col]

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

In [ ]:
# Create a production pipeline
production_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestRegressor(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    ))
])

print("Training production model...")
production_pipeline.fit(X_train, y_train)
print("✅ Model trained successfully!")

# Evaluate
y_pred = production_pipeline.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"\n📊 Model Performance:")
print(f"RMSE: ${rmse:,.2f}")
print(f"R²: {r2:.4f}")

### 2️⃣ Save Model - Multiple Methods

In [ ]:
import os

# Create models directory
os.makedirs('models', exist_ok=True)

# Method 1: Pickle (Python's built-in serialization)
print("💾 Saving model with Pickle...")
with open('models/model_pickle.pkl', 'wb') as f:
    pickle.dump(production_pipeline, f)
print("✅ Saved: models/model_pickle.pkl")

# Method 2: Joblib (recommended for sklearn, more efficient)
print("\n💾 Saving model with Joblib...")
joblib.dump(production_pipeline, 'models/model_joblib.pkl')
print("✅ Saved: models/model_joblib.pkl")

# Check file sizes
pickle_size = os.path.getsize('models/model_pickle.pkl') / 1024  # KB
joblib_size = os.path.getsize('models/model_joblib.pkl') / 1024  # KB

print(f"\n📦 File Sizes:")
print(f"Pickle: {pickle_size:.2f} KB")
print(f"Joblib: {joblib_size:.2f} KB")
print(f"\n💡 Joblib is typically more efficient for sklearn models")

### 3️⃣ Save Model Metadata

In [ ]:
# Create comprehensive metadata
metadata = {
    'model_info': {
        'name': 'Housing Price Predictor',
        'version': '1.0.0',
        'algorithm': 'Random Forest Regressor',
        'created_date': datetime.now().isoformat(),
        'author': 'ML Engineer'
    },
    'training_info': {
        'training_samples': len(X_train),
        'test_samples': len(X_test),
        'features': feature_cols,
        'target': target_col
    },
    'performance': {
        'test_rmse': float(rmse),
        'test_r2': float(r2),
        'train_rmse': float(np.sqrt(mean_squared_error(
            y_train, production_pipeline.predict(X_train)
        )))
    },
    'hyperparameters': {
        'n_estimators': 100,
        'max_depth': 10,
        'random_state': 42
    },
    'preprocessing': {
        'scaler': 'StandardScaler',
        'missing_value_strategy': 'median_imputation'
    }
}

# Save metadata
with open('models/model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=4)

print("✅ Metadata saved: models/model_metadata.json")
print("\n📄 Metadata Preview:")
print(json.dumps(metadata, indent=2))

### 4️⃣ Load and Verify Model

In [ ]:
# Load model
print("📂 Loading model from disk...")
loaded_model = joblib.load('models/model_joblib.pkl')
print("✅ Model loaded successfully!")

# Verify model works
print("\n🔍 Verifying model...")
test_prediction = loaded_model.predict(X_test[:5])
actual_values = y_test.iloc[:5].values

print("\nSample Predictions:")
for i, (pred, actual) in enumerate(zip(test_prediction, actual_values), 1):
    print(f"  Sample {i}: Predicted=${pred:,.0f}, Actual=${actual:,.0f}, "
          f"Error=${abs(pred-actual):,.0f}")

# Verify performance matches
y_pred_loaded = loaded_model.predict(X_test)
rmse_loaded = np.sqrt(mean_squared_error(y_test, y_pred_loaded))

print(f"\n✅ Performance Verification:")
print(f"Original RMSE: ${rmse:,.2f}")
print(f"Loaded RMSE: ${rmse_loaded:,.2f}")
print(f"Match: {np.isclose(rmse, rmse_loaded)}")

## Part 2: Creating Prediction Functions
### 5️⃣ Build Prediction Interface

In [ ]:
class HousePricePredictor:
    """
    Production-ready house price prediction class.
    
    Handles model loading, validation, and predictions.
    """
    
    def __init__(self, model_path='models/model_joblib.pkl', 
                 metadata_path='models/model_metadata.json'):
        """Initialize predictor with model and metadata."""
        self.model = joblib.load(model_path)
        
        with open(metadata_path, 'r') as f:
            self.metadata = json.load(f)
        
        self.feature_names = self.metadata['training_info']['features']
        print(f"✅ Loaded model version: {self.metadata['model_info']['version']}")
    
    def validate_input(self, input_data):
        """Validate input data format and values."""
        # Check if all required features are present
        missing_features = set(self.feature_names) - set(input_data.keys())
        if missing_features:
            raise ValueError(f"Missing features: {missing_features}")
        
        # Check for valid ranges (example)
        if input_data.get('median_income', 0) < 0:
            raise ValueError("median_income must be positive")
        
        return True
    
    def predict_single(self, input_data):
        """
        Make prediction for a single house.
        
        Args:
            input_data (dict): Dictionary with feature values
            
        Returns:
            dict: Prediction result with confidence interval
        """
        # Validate input
        self.validate_input(input_data)
        
        # Convert to DataFrame
        df = pd.DataFrame([input_data])[self.feature_names]
        
        # Make prediction
        prediction = self.model.predict(df)[0]
        
        # Calculate confidence interval (simplified)
        rmse = self.metadata['performance']['test_rmse']
        confidence_interval = (prediction - 2*rmse, prediction + 2*rmse)
        
        return {
            'predicted_price': float(prediction),
            'confidence_interval_95': {
                'lower': float(confidence_interval[0]),
                'upper': float(confidence_interval[1])
            },
            'model_version': self.metadata['model_info']['version'],
            'timestamp': datetime.now().isoformat()
        }
    
    def predict_batch(self, input_dataframe):
        """
        Make predictions for multiple houses.
        
        Args:
            input_dataframe (pd.DataFrame): DataFrame with features
            
        Returns:
            pd.DataFrame: Original data with predictions
        """
        predictions = self.model.predict(input_dataframe[self.feature_names])
        result = input_dataframe.copy()
        result['predicted_price'] = predictions
        return result
    
    def get_model_info(self):
        """Return model information."""
        return self.metadata['model_info']

# Test the predictor
predictor = HousePricePredictor()
print("\n📊 Model Info:")
print(json.dumps(predictor.get_model_info(), indent=2))

In [ ]:
# Test single prediction
sample_house = {
    'longitude': -122.23,
    'latitude': 37.88,
    'housing_median_age': 41.0,
    'total_rooms': 880.0,
    'total_bedrooms': 129.0,
    'population': 322.0,
    'households': 126.0,
    'median_income': 8.3252
}

print("🏠 Making prediction for sample house...")
result = predictor.predict_single(sample_house)

print("\n📊 Prediction Result:")
print(json.dumps(result, indent=2))
print(f"\n💰 Predicted Price: ${result['predicted_price']:,.0f}")
print(f"📈 95% Confidence Interval: "
      f"${result['confidence_interval_95']['lower']:,.0f} - "
      f"${result['confidence_interval_95']['upper']:,.0f}")

## Part 3: Creating a Simple REST API
### 6️⃣ Flask API Implementation

In [ ]:
# Save Flask API code to a file
flask_api_code = '''
from flask import Flask, request, jsonify
import joblib
import pandas as pd
import json
from datetime import datetime

# Initialize Flask app
app = Flask(__name__)

# Load model and metadata
model = joblib.load('models/model_joblib.pkl')
with open('models/model_metadata.json', 'r') as f:
    metadata = json.load(f)

feature_names = metadata['training_info']['features']

@app.route('/', methods=['GET'])
def home():
    """API home endpoint."""
    return jsonify({
        'message': 'House Price Prediction API',
        'version': metadata['model_info']['version'],
        'endpoints': {
            '/': 'API information',
            '/predict': 'Make single prediction (POST)',
            '/batch_predict': 'Make batch predictions (POST)',
            '/model_info': 'Get model information (GET)',
            '/health': 'Health check (GET)'
        }
    })

@app.route('/predict', methods=['POST'])
def predict():
    """Single prediction endpoint."""
    try:
        # Get input data
        data = request.get_json()
        
        # Validate features
        missing = set(feature_names) - set(data.keys())
        if missing:
            return jsonify({'error': f'Missing features: {list(missing)}'}), 400
        
        # Make prediction
        df = pd.DataFrame([data])[feature_names]
        prediction = model.predict(df)[0]
        
        # Calculate confidence interval
        rmse = metadata['performance']['test_rmse']
        
        return jsonify({
            'predicted_price': float(prediction),
            'confidence_interval_95': {
                'lower': float(prediction - 2*rmse),
                'upper': float(prediction + 2*rmse)
            },
            'model_version': metadata['model_info']['version'],
            'timestamp': datetime.now().isoformat()
        })
    
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/batch_predict', methods=['POST'])
def batch_predict():
    """Batch prediction endpoint."""
    try:
        # Get input data (list of dictionaries)
        data = request.get_json()
        
        if not isinstance(data, list):
            return jsonify({'error': 'Input must be a list of objects'}), 400
        
        # Make predictions
        df = pd.DataFrame(data)[feature_names]
        predictions = model.predict(df)
        
        return jsonify({
            'predictions': predictions.tolist(),
            'count': len(predictions),
            'model_version': metadata['model_info']['version'],
            'timestamp': datetime.now().isoformat()
        })
    
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/model_info', methods=['GET'])
def model_info():
    """Get model information."""
    return jsonify(metadata)

@app.route('/health', methods=['GET'])
def health():
    """Health check endpoint."""
    return jsonify({
        'status': 'healthy',
        'timestamp': datetime.now().isoformat()
    })

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)
'''

# Save to file
with open('models/app.py', 'w') as f:
    f.write(flask_api_code)

print("✅ Flask API code saved to: models/app.py")
print("\n📝 To run the API:")
print("1. Install Flask: pip install flask")
print("2. Run: python models/app.py")
print("3. API will be available at: http://localhost:5000")

### 7️⃣ API Testing Examples

In [ ]:
# Create example API requests
api_examples = {
    'single_prediction': {
        'method': 'POST',
        'endpoint': '/predict',
        'body': sample_house,
        'curl_example': f'''curl -X POST http://localhost:5000/predict \\
  -H "Content-Type: application/json" \\
  -d '{json.dumps(sample_house)}'
'''
    },
    'batch_prediction': {
        'method': 'POST',
        'endpoint': '/batch_predict',
        'body': [sample_house, sample_house],
        'curl_example': f'''curl -X POST http://localhost:5000/batch_predict \\
  -H "Content-Type: application/json" \\
  -d '{json.dumps([sample_house, sample_house])}'
'''
    },
    'model_info': {
        'method': 'GET',
        'endpoint': '/model_info',
        'curl_example': 'curl http://localhost:5000/model_info'
    },
    'health_check': {
        'method': 'GET',
        'endpoint': '/health',
        'curl_example': 'curl http://localhost:5000/health'
    }
}

# Save examples
with open('models/api_examples.json', 'w') as f:
    json.dump(api_examples, f, indent=2)

print("✅ API examples saved to: models/api_examples.json")
print("\n📚 Example API Calls:\n")

for name, example in api_examples.items():
    print(f"\n{name.upper().replace('_', ' ')}:")
    print(example['curl_example'])

## Part 4: Production Considerations
### 8️⃣ Model Versioning

In [ ]:
import shutil
from pathlib import Path

class ModelVersionManager:
    """
    Manage multiple versions of trained models.
    """
    
    def __init__(self, base_dir='models/versions'):
        self.base_dir = Path(base_dir)
        self.base_dir.mkdir(parents=True, exist_ok=True)
    
    def save_version(self, model, metadata, version):
        """Save a new model version."""
        version_dir = self.base_dir / f"v{version}"
        version_dir.mkdir(exist_ok=True)
        
        # Save model
        model_path = version_dir / 'model.pkl'
        joblib.dump(model, model_path)
        
        # Save metadata
        metadata['model_info']['version'] = version
        metadata_path = version_dir / 'metadata.json'
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
        
        print(f"✅ Saved model version {version}")
        return version_dir
    
    def load_version(self, version):
        """Load a specific model version."""
        version_dir = self.base_dir / f"v{version}"
        
        if not version_dir.exists():
            raise ValueError(f"Version {version} not found")
        
        model = joblib.load(version_dir / 'model.pkl')
        with open(version_dir / 'metadata.json', 'r') as f:
            metadata = json.load(f)
        
        return model, metadata
    
    def list_versions(self):
        """List all available versions."""
        versions = []
        for version_dir in self.base_dir.glob('v*'):
            if version_dir.is_dir():
                with open(version_dir / 'metadata.json', 'r') as f:
                    metadata = json.load(f)
                versions.append({
                    'version': metadata['model_info']['version'],
                    'created': metadata['model_info']['created_date'],
                    'performance': metadata['performance']
                })
        return versions
    
    def set_production(self, version):
        """Set a version as production."""
        version_dir = self.base_dir / f"v{version}"
        production_dir = self.base_dir.parent / 'production'
        
        # Remove old production
        if production_dir.exists():
            shutil.rmtree(production_dir)
        
        # Copy version to production
        shutil.copytree(version_dir, production_dir)
        
        print(f"✅ Set version {version} as production")

# Test version manager
vm = ModelVersionManager()
vm.save_version(production_pipeline, metadata, '1.0.0')
vm.set_production('1.0.0')

print("\n📦 Available Versions:")
for v in vm.list_versions():
    print(f"  Version {v['version']}: R²={v['performance']['test_r2']:.4f}")

### 9️⃣ Monitoring and Logging

In [ ]:
import logging
from collections import defaultdict

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('models/prediction.log'),
        logging.StreamHandler()
    ]
)

logger = logging.getLogger('HousePricePredictor')

class MonitoredPredictor(HousePricePredictor):
    """
    Predictor with monitoring and logging capabilities.
    """
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.prediction_stats = defaultdict(list)
        logger.info(f"Initialized predictor v{self.metadata['model_info']['version']}")
    
    def predict_single(self, input_data):
        """Make prediction with logging."""
        try:
            # Log request
            logger.info(f"Prediction request received")
            
            # Make prediction
            result = super().predict_single(input_data)
            
            # Track statistics
            self.prediction_stats['predictions'].append(result['predicted_price'])
            self.prediction_stats['timestamps'].append(result['timestamp'])
            
            # Log success
            logger.info(f"Prediction successful: ${result['predicted_price']:,.0f}")
            
            return result
        
        except Exception as e:
            logger.error(f"Prediction failed: {str(e)}")
            raise
    
    def get_stats(self):
        """Get prediction statistics."""
        if not self.prediction_stats['predictions']:
            return {'message': 'No predictions yet'}
        
        predictions = self.prediction_stats['predictions']
        return {
            'total_predictions': len(predictions),
            'average_prediction': np.mean(predictions),
            'min_prediction': np.min(predictions),
            'max_prediction': np.max(predictions),
            'std_prediction': np.std(predictions)
        }

# Test monitored predictor
monitored_predictor = MonitoredPredictor()

# Make some predictions
for i in range(5):
    result = monitored_predictor.predict_single(sample_house)

print("\n📊 Prediction Statistics:")
stats = monitored_predictor.get_stats()
print(json.dumps(stats, indent=2, default=str))

### 🔟 Creating Deployment Documentation

In [ ]:
deployment_docs = '''
# House Price Prediction Model - Deployment Guide

## Model Information
- **Model Name**: Housing Price Predictor
- **Version**: 1.0.0
- **Algorithm**: Random Forest Regressor
- **Performance**: R² = {r2:.4f}, RMSE = ${rmse:,.2f}

## Quick Start

### 1. Installation
```bash
pip install -r requirements.txt
```

### 2. Run API Server
```bash
python models/app.py
```

### 3. Make Predictions
```python
import requests

# Single prediction
response = requests.post(
    'http://localhost:5000/predict',
    json={sample_data}
)
print(response.json())
```

## API Endpoints

### GET /
API information and available endpoints

### POST /predict
Make single prediction
- **Input**: JSON object with house features
- **Output**: Predicted price with confidence interval

### POST /batch_predict
Make batch predictions
- **Input**: Array of JSON objects
- **Output**: Array of predictions

### GET /model_info
Get model metadata and performance metrics

### GET /health
Health check endpoint

## Required Features
1. longitude
2. latitude
3. housing_median_age
4. total_rooms
5. total_bedrooms
6. population
7. households
8. median_income

## Production Deployment

### Docker Deployment
```dockerfile
FROM python:3.10-slim
WORKDIR /app
COPY requirements.txt .
RUN pip install -r requirements.txt
COPY models/ models/
CMD ["python", "models/app.py"]
```

### Environment Variables
- `MODEL_PATH`: Path to model file
- `PORT`: API port (default: 5000)
- `LOG_LEVEL`: Logging level (default: INFO)

## Monitoring

### Metrics to Track
1. **Prediction Latency**: Response time
2. **Prediction Distribution**: Monitor for drift
3. **Error Rate**: Failed predictions
4. **Model Performance**: Compare predictions vs actuals

### Logging
All predictions are logged to `models/prediction.log`

## Model Retraining

### When to Retrain
- Performance degradation detected
- New data available
- Scheduled (e.g., monthly)

### Retraining Process
1. Collect new data
2. Train new model
3. Validate performance
4. Save as new version
5. A/B test before full deployment

## Troubleshooting

### Common Issues
1. **Missing Features**: Ensure all 8 features are provided
2. **Invalid Values**: Check for negative values
3. **Model Not Found**: Verify model path

## Support
For issues or questions, contact: ml-team@example.com
'''.format(r2=r2, rmse=rmse, sample_data=json.dumps(sample_house, indent=2))

# Save documentation
with open('models/DEPLOYMENT.md', 'w') as f:
    f.write(deployment_docs)

print("✅ Deployment documentation saved to: models/DEPLOYMENT.md")

## 📊 Key Takeaways

### Deployment Best Practices:

#### 1. **Model Saving**:
✅ Use joblib for sklearn models (more efficient)  
✅ Save complete pipelines, not just models  
✅ Include preprocessing steps  
✅ Version your models  
✅ Save metadata with models  

#### 2. **API Design**:
✅ RESTful endpoints  
✅ Input validation  
✅ Error handling  
✅ Health checks  
✅ Documentation  

#### 3. **Production Considerations**:
✅ Logging and monitoring  
✅ Model versioning  
✅ A/B testing capability  
✅ Rollback strategy  
✅ Performance metrics  

#### 4. **Monitoring**:
✅ Prediction latency  
✅ Error rates  
✅ Model drift detection  
✅ Data quality checks  
✅ Resource usage  

### Deployment Checklist:

- [ ] Model trained and validated
- [ ] Model saved with metadata
- [ ] Prediction function tested
- [ ] API endpoints implemented
- [ ] Input validation added
- [ ] Error handling implemented
- [ ] Logging configured
- [ ] Documentation created
- [ ] Health checks added
- [ ] Monitoring set up
- [ ] Version control in place
- [ ] Rollback strategy defined

### Common Deployment Patterns:

| Pattern | Use Case | Pros | Cons |
|---------|----------|------|------|
| **Batch** | Scheduled predictions | Simple, efficient | Not real-time |
| **REST API** | On-demand predictions | Flexible, scalable | Requires server |
| **Streaming** | Real-time data | Low latency | Complex setup |
| **Embedded** | Mobile/Edge | No network needed | Limited resources |

### Tools and Frameworks:

**API Frameworks:**
- Flask (simple, lightweight)
- FastAPI (modern, fast)
- Django REST (full-featured)

**Deployment Platforms:**
- Docker (containerization)
- Kubernetes (orchestration)
- AWS SageMaker (managed ML)
- Google Cloud AI Platform
- Azure ML

**Monitoring:**
- Prometheus + Grafana
- ELK Stack (Elasticsearch, Logstash, Kibana)
- MLflow
- Weights & Biases

### Next Steps:
1. Deploy to cloud platform (AWS, GCP, Azure)
2. Implement CI/CD pipeline
3. Add model monitoring dashboard
4. Set up automated retraining
5. Implement A/B testing framework